In [1]:
import numpy as np
import pandas as pd
import tushare as ts
from utils import *
print(ts.__version__)

1.2.84


In [2]:
ts.set_token('1f2f092156dfe775a88e478f3a54565e0adab29246c83814e95fa0dd')
pro = ts.pro_api()

## 过滤概念板块

In [3]:
# 过滤概念板块
concept_stocks = pd.read_csv("./data/concept_stocks.csv")  # (34444, 6)
print(concept_stocks.shape, concept_stocks.columns, concept_stocks.tail())

concept_groups = concept_stocks.groupby("concept_name", as_index=False)  # 873 groups, indexed from TS0 to TS879
print(len(concept_groups))
#for name, group in concept_groups:
#    if len(group) > 223 or len(group)<=2:
#        print(name, len(group))
concept_groups = concept_groups.filter(lambda x: len(x)<=223 and len(x)>=3)
print(len(concept_groups), len(concept_groups.groupby("id")))  # .get_group("TS0")
concept_groups.to_csv("./data/concept_stocks_filtered_223_3.csv", index=False)  # 23928, 824
# {name:gdf for name,gdf in df.groupby('Declarer') if len(gdf) >= 5}

(34444, 6) Index(['id', 'concept_name', 'ts_code', 'name', 'in_date', 'out_date'], dtype='object')           id concept_name    ts_code  name   in_date  out_date
34439  TS879         3D打印  603112.SH  华翔股份  20210205       NaN
34440  TS879         3D打印  603928.SH  兴业股份  20210205       NaN
34441  TS879         3D打印  603977.SH  国泰集团  20210205       NaN
34442  TS879         3D打印  605376.SH  博迁新材  20210205       NaN
34443  TS879         3D打印  688333.SH   铂力特  20210205       NaN
873
23928 824


## 建图代码

In [4]:
# 建图
#  edge_index: [2, num_edges] 
def get_rind_map(indsty):
	groups = pd.read_csv(indsty).groupby('id', as_index=False)
	rind_map = {id: group["ts_code"].values.tolist() for id, group in groups}
	return rind_map


def set_edge(mat, stocks, stock_id_map):
	# stock-stock edges
	for i in range(len(stocks)):
		for j in range(i+1, len(stocks)):
			s = stock_id_map.get(stocks[i], -1)
			e = stock_id_map.get(stocks[j], -1)
			if s==-1 or e==-1: continue
			mat[s,e] = mat[e,s] = 1


def set_heteredge(idx, mat, stocks, stock_id_map):
	# stock-industry edges
	e = idx + len(stock_id_map)
	for i in range(len(stocks)):
		s = stock_id_map.get(stocks[i], -1)
		if s==-1 or e==-1: continue
		mat[s,e] = mat[e,s] = 1


def build_hetergraph(indsty, return_adj, stock_id_map):
	snode_nums = len(stock_id_map)
	rind_map = get_rind_map(indsty)

	mat = torch.eye(snode_nums+len(rind_map.keys()))  # add self loop

	for i, (ind, stocks) in enumerate(rind_map.items()):
		set_heteredge(i, mat, stocks, stock_id_map)
		
	edge = mat.nonzero()
	return snode_nums, snode_nums+len(rind_map.keys()), mat if return_adj == True else edge.transpose(0, 1)


def build_graph(indsty, return_adj, stock_id_map):
	node_nums = len(stock_id_map)
	rind_map = get_rind_map(indsty)

	mat = torch.eye(node_nums)  # add self loop

	for ind, stocks in rind_map.items():
		set_edge(mat, stocks, stock_id_map)
		
	edge = mat.nonzero()
	return mat if return_adj == True else edge.transpose(0, 1)

## 概念板块建图

In [ ]:

stock_list_path = "./data/stock_codes_1931.txt"
stock_id_map = get_stock_id_mapping(stock_list_path)
adj = build_graph("./data/concept_stocks_filtered_223_3.csv", True, stock_id_map)

print(adj.shape)

In [ ]:
np.save("./data/concepts_graph_1931_233_3.npy", adj.data)

In [ ]:
# 给定0-1邻接矩阵，画图，x是node degree, y是节点数目/比例
from matplotlib import pyplot as plt

adj = adj - adj.diag()  # remove self-loop
degree_cnt = [0 for i in range(len(adj))]
edges = 0
for i in range(len(adj)):
    d = int(sum(adj[i]))
    edges += d
    degree_cnt[d] += 1

print(edges / (len(adj)**2))
plt.plot([i for i in range(1, 600)], degree_cnt[1:600])  # x, y
plt.xlabel("node degree")
plt.ylabel("node cnt")
plt.show()